### Урок 5. Рекуррентные нейронные сети

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Dropout, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
# По идее это должно быть про датасет с IMDB но он про англоязычные кино-рецензии, это не так интересно
# Для курсового по предыдущему курсу я стянул с нашего "кинопоиск-а" 1700 рецензий на русском языке
# Разумеется они там тоже размечены - на сайте стоит оценка (не бинарная) - негативная, нейтральная, положительная

data = pd.read_csv('kinopoisk.csv')
data.head()

,Unnamed: 0,grade,text
0,0,1.0,Гениальный студент Ягами Лайт с помощью тетрад...
1,1,1.0,"Как известно, в СССР кино было делом государст..."
2,2,1.0,На фоне многих других фильмов Марвел первые д...
3,3,1.0,"Вот и завершила я просмотр первой фазы, вводно..."
4,4,1.0,"Так получилось, что с актером Дэнилом Рэдклифф..."


In [4]:
data.shape

(1710, 3)

In [5]:
data['grade'].value_counts(normalize=False)

1.0    1249
0.5     253
0.0     208
Name: grade, dtype: int64

In [6]:
# Неудачный момент состоит в том, что бОльшая часть рецензий положительные (1249), 
# нейтральных и негативных примерно одинаковое количество и их мало (253 и 208)
# Такой вот несбалансированный датасет получился

In [7]:
# Не знаю - есть ли в Keras то, что сможет посчитать количество слов в корпусе текста всех рецензий - 
# Воспользуемся sklearn-овским

from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer()

In [9]:
data_tf = tfidf.fit_transform(data.text).toarray()
data_tf.shape

(1710, 78568)

In [10]:
# 78568 слов
max_features = data_tf.shape[1]

In [11]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data.text)
X = np.array(tokenizer.texts_to_sequences(data.text))
Y = to_categorical(data.grade)

In [12]:
Y.shape

(1710, 2)

In [13]:
# странно - категории 3 - а размерность получилась 2... 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data.grade)

LabelEncoder()

In [14]:
grades = encoder.transform(data.grade)
grades

array([2, 2, 2, ..., 2, 2, 1])

In [15]:
Y = to_categorical(grades)
Y.shape

(1710, 3)

In [16]:
# Вот - так другое дело.. глаз да глаз )
categories = ['negative', 'neutral', 'positive']

In [17]:
# Делимся на трейн и тест
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [18]:
x_train.shape

(1368,)

In [19]:
# Кот наплакал )))

In [20]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=128, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=128, padding='post')

In [21]:
x_train_pad.shape

(1368, 128)

In [22]:
from tensorflow.keras.metrics import Precision, Recall
model = Sequential()
model.add(Embedding(max_features, 512))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [407]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, None, 512)         40226816  
_________________________________________________________________
lstm_37 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_54 (Dense)             (None, 8192)              4202496   
_________________________________________________________________
dropout_6 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dense_56 (Dense)             (None, 3)                 1539      
Total params: 50,724,867
Trainable params: 50,724,867
Non-trainable params: 0
_________________________________________

In [408]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy', Precision(), Recall()])

In [409]:
model.fit(x_train_pad, y_train, batch_size=50, epochs=1, 
                    validation_data=(x_test_pad, y_test))

28/28 [==============================] - 74s 3s/step - loss: 0.7639 - accuracy: 0.7061 - precision_20: 0.7522 - recall_20: 0.6389 - val_loss: 0.6205 - val_accuracy: 0.7398 - val_precision_20: 0.8144 - val_recall_20: 0.6930


In [410]:
# Всё вроде как-то получилось...

In [411]:
test_review = '''Знаете, Михаила Сегала игнорировать невозможно - 'Рассказы' это знаменательный фильм в российском кино 10-х. Не потому что фильм хороший (не плохой, конечно, тоже), не потому что он важный, актуальный или яркий, а потому что ничего подобного рядом просто нет. В целом, он конечно не прыгает выше головы того культурного пласта, который называется современным российским кино, но это настолько поехавший и противоречивый его представитель, что посмотрев его однажды - я до сих пор его помню. Впечатления от него сравнимы с тем, что если бы на сцену какого-нибудь крутого западного клуба вместо тогдашнего Луи Си Кея вышел бы Евгений Ваганович - никто бы его там не знал, и все ждали бы чего-нибудь смешного и современного, но вот он вышел и начал шутить все те же свои шутки, которые устарели еще лет десять назад. Но никто не фукает, не кидает в него помидоры - все просто заворожены и не понимают, что происходит: это не смешно, неактуально, невероятно вымучено, но... необычно и даже чуть-чуть увлекательно. И все зрители ждут: он же щас что-то сделает? Что-то классное? Что-то крутое? Вот же почти!.. .Но нет, все еще нудно. Порой зал взрывается нервным смешком, потому что когда вы долго ожидаете панчлайнов, а их попросту нет, даже какая-нибудь невпопад брошенная фразочка про Холокост возымеет невероятный комический эффект. Именно вот подобный эффект 'да что это вообще такое?' вызывают 'Рассказы'. И 'Глубже'. Потому что 'Глубже' - это 'Рассказы', просто полный метр, а не альманах. 

Вообще-то это настолько 'Рассказы', что тут даже есть длинная вымученная шутка из 'Рассказов', перенесенная точь-в-точь, с одним и тем же посылом, в одном и том же контексте с одним и тем же исполнением. И даже тут продолжаешь пребывать в состоянии 'да что это вообще такое?', потому что с одной стороны это может быть перфоманс, троллящий знающих о 'Рассказах' зрителей намеренным самоцитированием, или это может быть объединением фильмов в 'киновселенную'. Но в то же время это может быть и просто лень автора, или, что намного хуже, то, что он считает эту шутку невероятно хорошей, чтобы повторить ее дважды (а она длинная!). Но если это так, почему никто не подошел к режиссеру, не дал ему пенделя под зад и не прокричал: 'Чел, она не была смешной уже в 2012-ом!'. И это ни фига не сатира, не ирония, не постирония, не метаирония - это тот жанр юмора, когда Путин на КВН приезжает и квн-щики типа пытаются про него пошутить - очень смешно, животик прям надрывается.

И это очень забавно, что Михаил Сегал назвал 'Глубже' фильм, который крайне поверхностно пытается отразить современную реальность, пробежавшись по макушкам собственных представлений о мире, нарочно утрируя их, потому что... это так глубоко! Просто не режиссер, а монстр, обитающий на глубине, которую не постигнет человеческий разум и где надо панамку одевать, чтобы солнышко голову не напекло. 

На самом деле, я так и не понял, метафорой чего является порно тут, поэтому я пошел читать свою любимую пояснительную бригаду - Антона Долина (не то чтобы я его фанат, но он реально хорошо объясняет непонятные моменты) и прям рад за себя - Долин, тоже видимо не понял, к чему это. Но он напрямую дал мысль, которая довольно легко объясняет почему 'Глубже' именно сделан именно так (да и 'Рассказы' и может быть остальное, но я больше ничего не смотрел) - Сегал думает, что он Чехов. Признаюсь - не очень люблю Чехова, поэтому есть шанс, что в этом Сегал как раз очень хорош и зайдет фанатам пострадать над перипетиями девятнадцатого века и 'вечной российской тоске по Хорошему Вкусу и Высокому Искусству' (это дословная цитата из Долина!). 

С другой стороны, как забавно то, что квинтэссенцией Творчества нашего недочехова стала сцена из рассказов, где мужик истерит и докапывается до девушки (та же Любовь Аксенова в абсолютно такой же роли, что и тут), что та типа не знает... чет там про войну. Я все думал, почему это там так кринжово и окончательно понял именно при просмотре 'Глубже' - странновато наблюдать, что фильм упрекает кого-то в незнании реальности, когда сам режиссер буквально отказывается ее принимать, замещая слащавым мифом и утрированным слепком, отдаленно напоминающим действительность людям, не покидающим свои участки с двухметровыми глухими заборами и наблюдающим за миром из панорамных окон ресторанчиков и с фестивалей в торговых центрах. И из фильмов Сегала это очевидно и это не то, чтобы плохо, просто его внезапные истерические откровения и обращения к людям (которое тут тоже накрывает героя!), выглядят просто нелепо на фоне того глянцевого мира, который он сам создает. Или наиболее смешно это в сцене, где герой Паля раздевает актеров театра на сцене, пытаясь таким образом заставить их играть 'по-настоящему', сломать их 'плоскость' и раскрыть 'глубину'. И при этом сам фильм не может показать обнаженку! Да, и всю эту вымученную сцену они там стоят прикрывая пах и грудь. Я не знаю, в чем тут причина - возможно Сегал придерживается очень пуританских взглядов, возможно так настояли продюсеры, или они хотели получить '6+' (не получилось!), но это берет особую планку лицемерия - кричать о чем-то и тут же, в то же самое время быть неспособным самому этому соответствовать. Поэтому, да, в это просто весь Сегал: человек упрекающий молодое поколение в незнании истории, но при этом сам проецирующий мир поней и единорогов; человек требующий откровенности, но не собирающийся ее показывать. И да, это уровень тех ребят в ВК, которые так злы, так экспрессивны и так обругают вас матом, что... будут в матерных словах менять буковки, чтобы их не забанили! 

В принципе, нет, мне даже чем-то нравится Михаил Сегал - в череде безликих незапоминающихся режиссеров у него личность-то есть и кино его авторское, и даже уникальное и иногда даже забавно наблюдать, как в одном фильме один человек борется между горячим желанием высказаться и паническим ужасом это сделать.'''

In [412]:
test_review

"Знаете, Михаила Сегала игнорировать невозможно - 'Рассказы' это знаменательный фильм в российском кино 10-х. Не потому что фильм хороший (не плохой, конечно, тоже), не потому что он важный, актуальный или яркий, а потому что ничего подобного рядом просто нет. В целом, он конечно не прыгает выше головы того культурного пласта, который называется современным российским кино, но это настолько поехавший и противоречивый его представитель, что посмотрев его однажды - я до сих пор его помню. Впечатления от него сравнимы с тем, что если бы на сцену какого-нибудь крутого западного клуба вместо тогдашнего Луи Си Кея вышел бы Евгений Ваганович - никто бы его там не знал, и все ждали бы чего-нибудь смешного и современного, но вот он вышел и начал шутить все те же свои шутки, которые устарели еще лет десять назад. Но никто не фукает, не кидает в него помидоры - все просто заворожены и не понимают, что происходит: это не смешно, неактуально, невероятно вымучено, но... необычно и даже чуть-чуть увл

In [413]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))

In [416]:
pred = model.predict(np.asarray(test_tokenized))

In [417]:
categories[np.argmax(pred[0])]

'positive'

In [418]:
# Не попала.. (( модель думает, что это позитивная рецензия - на кинопоиске она классифицирована как негативная

In [419]:
test_review = '''Новый фильм Михаила Сегала, творца таких шедевров как «Рассказы», «Овцы» и «Слоны могут играть в футбол». Т.е. он снимает кино не просто не для всех, но даже те немногочисленные его поклонники, которым по итогу тоже не нравится или не понятно, начинают издеваться друг над другом. Ведь нельзя же не понимать Сегала!

Рейтинг КП 6.6, что для фильма российского производства последних лет, довольно неплохо. В чём же успех?

В порно! В описании фильма на целых 2 строчки слово «порно» фигурирует 2 раза и один раз с определением «глубокое». Отличный маркетинговый ход, чтобы привлечь зрителей. Российское кино и так сплошь и рядом снимается в жанре чернушной порнографии, но если об этом заявить ещё и в анонсе, то успех обеспечен.

Сюжет прост как 3 копейки. Театральный режиссёр, Роман Петрович (Александр Паль) пытается ставить пьесы по системе Станиславского, но актёры не разделяют его глубоко подхода и старательно изображают на сцене дерево, поэтому он выбивается из плана и попадает под увольнение. Глубокий подход главного героя нигде не позволяет ему задерживаться надолго, пока он совершенно случайно не попадает в индустрию порно, где начинает работать также, как и раньше, по системе Станиславского.

И порно в его режиссуре получает невиданную доселе глубину. Но порно — это лишь отправная точка, чтобы двигаться дальше. Романа Петровича замечают на самом верху и настоятельно рекомендуют Чехова ставить в жанре, который у 'главного режиссёра страны' получается лучше всего. И эти его умения создатель фильма транслирует на всё в стране. В какой-то момент многие начинают задаваться вопросом: «Что за…?»

Если этот посыл не кажется вам достаточно глубоким, то Сегал на постере крупным шрифтом пишет «Глубже!», ну, т.е. смысл гораздо глубже, чем вам кажется. И тут люди начинают думать, размышлять, обсуждать, подводить основу под эту вакханалию. Лично у меня возник вопрос, если вы действительно хотите снять серьёзное глубокое кино, то зачем вы снимаете это и пытаетесь выдать его за то, чем оно на самом деле не является? Я отлично понимаю, что такое аллегория, гротеск и преувеличение, но сами художественные приёмы не могут заменить отсутствие содержания.

Итого тема порно не раскрыта. И я уж не знаю, почему Сегал считает, что он снимает что-то лучше, потому что жанр порно работает по системе Станиславского с самого своего зарождения и ему верят гораздо больше и чаще, чем серьёзному фильму «Глубже!». Так что сравнение получилось совсем не в пользу последнего. За что боролись, на то и напоролись.

Рекомендовать это глубокое произведение, не скажу, что искусства, не буду. Смотрите на свой страх и риск. На комедию это мало похоже, смеяться там негде. А искать смысл лучше в других работах, отнюдь не руки Сегала.'''

In [421]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'positive'

In [422]:
# Опять мимо

In [423]:
test_review = '''Общее впечатление: Перед нами Роман (Александр Паль) режиссер в театре. Так вышло, что Романа не понимают на работе, уж слишком глубоко он копает, по воли случая он знакомится с порно-индустрией и пытается вдохнуть в такое непросто направление глубокие чувства. Ведь так хочется смотреть на настоящую драму между людьми! И что самое удивительное, именно там, в порно, Роман находит свое призвание.

Заявленный жанр «Глубже!» — комедия. Но справляется лента с этим направлением 50/50. Есть достаточно много иронии и в сторону общества, и театра, но призрачная нить так и норовит выпрыгнуть из рук. Почему-то столь твердое плато в начале фильма разваливается по кускам, образуя скучноватую историю, завернутую в монотонный вымысел. Начало у ленты увлекательно и динамично проскальзывает вокруг ГГ, он не может найти себе место в жизни, занимаясь любимым делом, совсем не чувствует отдачи (в принципе как и многие). И только попав в порно-индустрию, происходят метаморфозы. Роман добивается того чего так давно хотел, не только раскрывая свой талант режиссера, но и находит финансовую стабильность. Именно первая часть фильма смешная, есть моменты курьезные и обыграны они вполне неплохо. То что должно подаваться серьезно, раскрывается комичной абсурдностью. Именно это и привлекает зрителя. Но как только герой добивается поставленной цели, а добивается он ее примерно за час к/ф, то интерес к просмотру тут же пропадает.

Примечательный каст — сильная сторона картины. Александр Паль, Любовь Аксенова, Семен Трескунов, Игорь Угольников, Игорь Верник, Сергей Бурунов. Все лица знакомые, но не замыленные. Безусловно, основное внимание на герое Паля, у Александра отлично получилось передать творческий образ. По сути, на Пале все и держится.

В итоге, что имеем? Оригинальность бесспорно интересна, но ее отражение в фильме порою хромает. Чего не хватило? Крепкого и устойчивого плато. Оправдывает ли свое название «Глубже!» — иногда смешно, но поистине хороший и действительно глубокий эпизод в фильме один — финал.
'''

In [426]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'positive'

In [427]:
# И здесь не туда

In [428]:
test_review = '''Интересно, сколько же людей честно заманит фильм порнхабовским дизайном постера, трейлером и комедийным синопсисом, ожидая увидеть аналог знаменитой ленты Кевина Смита. Ну или хотя бы попытку в 'Ночи в стиле буги', с тем же профессиональным настроем и желанием. История совсем не про это. Повезет заметившим Михаила Сегала как постановщика и автора сценария, а значит готовым к тому, что тематика порно станет лишь одной частью мощной мыслительной авторской конструкции. Да и нужно оно больше оказалось для рекламной кампании. 

Михаил Сегал безусловно непосредственный, авантюрный создатель контента, которого за данный фильм стоит упоминать и хвалить, даже учитывая то, что зайдет проект далеко не всем. Даже учитывая рефлексию творца и некоторые махровые, иконоподобные, штампы, въевшиеся уже лет 10 назад: если сценка в порнухе - то сантехник; если жизненное дно - то гопники, отжимающие телефон; если главный телеканал страны - то провластный первый. Да и когда там театр был символом и гордостью державы, способным утереть нос западу. Тем не менее, все эти непопулярные и изжившие элементы автор умудряется влить в единый глубокий флакон с порнографией, и заставить досмотреть, иногда восхищаясь полету фантазии. Моментами неуклюжесть и кринжовость постановки доходит до 'псковского порно', а порой очень точечно охватывает азарт. 

Александр Паль - это действительно хороший выбор главного героя, пусть он автоматически крадет градус серьезности у происходящего, но его образ, сотканный из постиронии выглядит органично. Хотя временами крадется мысль, что было бы, поменяй их с Лапенко местами, уж очень он колоритный персонаж. Паль так же удобен при визуальной аллюзии на Чехова, да и монотонность образа позволяет лучше считывать текст, не отвлекаясь на импульсы актера. Для политиков, тем более главного, выбор кажется не слишком удачным, Аксенова хороша опять же, в роли рекламного лица, Бурунов выглядит немного посторонним в истории, но эпизод свой определенно затащил. 

'Глубже!' - фильм-событие, фильм-высказывание, его очень хочется обсудить, но не очень пересмотреть. Спитч о культуре, завернутый в симпатичный, по-хорошему чеховский финал, в котором на поверхность всплывают яркие постулаты о том, что: классика - наше все, и решения многих проблем уже там есть, а мыслить и чувствовать действительно надо шире и глубже. Почему фильм зайдет и заходит не всем, пожалуй, главный вопрос. Михаил Сегал действительно намешал многовато: личная история успеха, ремейк порнографического жанра, карикатуры реальных личностей, проблемы современного творчества, политическая сатира, театральные этюды, и все это в постироничной, серо-монотонной подаче. Интересно, но весь этот фильдеперсовый дивертисмент ведет через долгую дорогу к не самым глубоким мыслям, и превращается в интенсивную кашу, ковыряться в которой возьмется не каждый. Уж точно не те, кто ассоциативно пришел на драйвовую, легкую комедию со взрослым рейтингом.'''

In [429]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'positive'

In [431]:
# Нет шансов похоже ) предсказывать правильно оно может только положительные ))

In [34]:
# Попробуем связать несколько рекуррентных слоев

from tensorflow.keras.layers import Bidirectional
model = Sequential()
model.add(Embedding(max_features, 128, mask_zero=True))
model.add(Bidirectional(LSTM(128, dropout = 0.2, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout=0.2)))
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [37]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy', Precision(), Recall()])

In [38]:
model.fit(x_train_pad, y_train, batch_size=50, epochs=2, 
                    validation_data=(x_test_pad, y_test))

Epoch 1/2
28/28 [==============================] - 25s 904ms/step - loss: 0.8168 - accuracy: 0.7061 - precision_4: 0.7251 - recall_4: 0.6440 - val_loss: 0.7340 - val_accuracy: 0.7398 - val_precision_4: 0.7439 - val_recall_4: 0.7135
Epoch 2/2
28/28 [==============================] - 24s 871ms/step - loss: 0.4794 - accuracy: 0.7792 - precision_4: 0.9032 - recall_4: 0.6820 - val_loss: 1.0763 - val_accuracy: 0.7398 - val_precision_4: 0.7523 - val_recall_4: 0.7281


In [497]:
test_review = '''Интересно, сколько же людей честно заманит фильм порнхабовским дизайном постера, трейлером и комедийным синопсисом, ожидая увидеть аналог знаменитой ленты Кевина Смита. Ну или хотя бы попытку в 'Ночи в стиле буги', с тем же профессиональным настроем и желанием. История совсем не про это. Повезет заметившим Михаила Сегала как постановщика и автора сценария, а значит готовым к тому, что тематика порно станет лишь одной частью мощной мыслительной авторской конструкции. Да и нужно оно больше оказалось для рекламной кампании. 

Михаил Сегал безусловно непосредственный, авантюрный создатель контента, которого за данный фильм стоит упоминать и хвалить, даже учитывая то, что зайдет проект далеко не всем. Даже учитывая рефлексию творца и некоторые махровые, иконоподобные, штампы, въевшиеся уже лет 10 назад: если сценка в порнухе - то сантехник; если жизненное дно - то гопники, отжимающие телефон; если главный телеканал страны - то провластный первый. Да и когда там театр был символом и гордостью державы, способным утереть нос западу. Тем не менее, все эти непопулярные и изжившие элементы автор умудряется влить в единый глубокий флакон с порнографией, и заставить досмотреть, иногда восхищаясь полету фантазии. Моментами неуклюжесть и кринжовость постановки доходит до 'псковского порно', а порой очень точечно охватывает азарт. 

Александр Паль - это действительно хороший выбор главного героя, пусть он автоматически крадет градус серьезности у происходящего, но его образ, сотканный из постиронии выглядит органично. Хотя временами крадется мысль, что было бы, поменяй их с Лапенко местами, уж очень он колоритный персонаж. Паль так же удобен при визуальной аллюзии на Чехова, да и монотонность образа позволяет лучше считывать текст, не отвлекаясь на импульсы актера. Для политиков, тем более главного, выбор кажется не слишком удачным, Аксенова хороша опять же, в роли рекламного лица, Бурунов выглядит немного посторонним в истории, но эпизод свой определенно затащил. 

'Глубже!' - фильм-событие, фильм-высказывание, его очень хочется обсудить, но не очень пересмотреть. Спитч о культуре, завернутый в симпатичный, по-хорошему чеховский финал, в котором на поверхность всплывают яркие постулаты о том, что: классика - наше все, и решения многих проблем уже там есть, а мыслить и чувствовать действительно надо шире и глубже. Почему фильм зайдет и заходит не всем, пожалуй, главный вопрос. Михаил Сегал действительно намешал многовато: личная история успеха, ремейк порнографического жанра, карикатуры реальных личностей, проблемы современного творчества, политическая сатира, театральные этюды, и все это в постироничной, серо-монотонной подаче. Интересно, но весь этот фильдеперсовый дивертисмент ведет через долгую дорогу к не самым глубоким мыслям, и превращается в интенсивную кашу, ковыряться в которой возьмется не каждый. Уж точно не те, кто ассоциативно пришел на драйвовую, легкую комедию со взрослым рейтингом.'''

In [499]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'neutral'

In [500]:
# Вау.. Угадало!

In [501]:
test_review = '''Честно говоря, героиня Лядовой намного интереснее главного героя. Ее хочется смотреть. Экстравагантного, экзистенциального персонажа - самого психа -нет. Не тянет он. А проблему секса в пожилом возрасте, с которой сталкивается мать героя -выглядит откровенно странно, искусственно и даже слегка уродливо, из-за специфической внешности персонажа Розы Хайрулиной. Она больше похожа на человека из космоса, нежели на женщину с типичными 'проевропейскими' проблемами. Не резонирует. Даже не вызывает интерес. Сериал не выглядит даже как самобытный изыск. Он выглядит как технологичная попытка выйти из сериального шаблона, но оттого создающая тот самый шаблон. Претензия на 'умное' кино не оправдана. Смотрела из любопытства, но с усилием. В конце уже просто на перемотке. Не цепляет.'''

In [503]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'neutral'

In [474]:
# Nice try... Теплее, но - нет.

In [523]:
model = Sequential()
model.add(Embedding(max_features, 128, mask_zero=True))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [524]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy', Precision(), Recall()])

In [525]:
model.fit(x_train_pad, y_train, batch_size=50, epochs=3, 
                    validation_data=(x_test_pad, y_test))

Epoch 1/3
28/28 [==============================] - 9s 311ms/step - loss: 0.8780 - accuracy: 0.7193 - precision_36: 0.7237 - recall_36: 0.4576 - val_loss: 0.7548 - val_accuracy: 0.7398 - val_precision_36: 0.7398 - val_recall_36: 0.7398
Epoch 2/3
28/28 [==============================] - 8s 282ms/step - loss: 0.6858 - accuracy: 0.7281 - precision_36: 0.7345 - recall_36: 0.7281 - val_loss: 0.6696 - val_accuracy: 0.7398 - val_precision_36: 0.7426 - val_recall_36: 0.7339
Epoch 3/3
28/28 [==============================] - 8s 297ms/step - loss: 0.3141 - accuracy: 0.8319 - precision_36: 0.9278 - recall_36: 0.7697 - val_loss: 0.8494 - val_accuracy: 0.7515 - val_precision_36: 0.7935 - val_recall_36: 0.7193


In [526]:
test_review = '''Честно говоря, героиня Лядовой намного интереснее главного героя. Ее хочется смотреть. Экстравагантного, экзистенциального персонажа - самого психа -нет. Не тянет он. А проблему секса в пожилом возрасте, с которой сталкивается мать героя -выглядит откровенно странно, искусственно и даже слегка уродливо, из-за специфической внешности персонажа Розы Хайрулиной. Она больше похожа на человека из космоса, нежели на женщину с типичными 'проевропейскими' проблемами. Не резонирует. Даже не вызывает интерес. Сериал не выглядит даже как самобытный изыск. Он выглядит как технологичная попытка выйти из сериального шаблона, но оттого создающая тот самый шаблон. Претензия на 'умное' кино не оправдана. Смотрела из любопытства, но с усилием. В конце уже просто на перемотке. Не цепляет.'''

In [528]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'neutral'

In [39]:
model = Sequential()
model.add(Embedding(max_features, 128, mask_zero=True))
model.add(Bidirectional(LSTM(256, dropout = 0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy', Precision(), Recall()])

In [542]:
model.fit(x_train_pad, y_train, batch_size=50, epochs=3, 
                    validation_data=(x_test_pad, y_test))

Epoch 1/3
28/28 [==============================] - 37s 1s/step - loss: 0.8408 - accuracy: 0.7061 - precision_40: 0.7263 - recall_40: 0.5877 - val_loss: 0.7069 - val_accuracy: 0.7398 - val_precision_40: 0.7398 - val_recall_40: 0.7398
Epoch 2/3
28/28 [==============================] - 40s 1s/step - loss: 0.6016 - accuracy: 0.7449 - precision_40: 0.7945 - recall_40: 0.7178 - val_loss: 0.6259 - val_accuracy: 0.7573 - val_precision_40: 0.8136 - val_recall_40: 0.7018
Epoch 3/3
28/28 [==============================] - 36s 1s/step - loss: 0.2492 - accuracy: 0.8692 - precision_40: 0.9124 - recall_40: 0.8143 - val_loss: 1.1021 - val_accuracy: 0.7515 - val_precision_40: 0.7683 - val_recall_40: 0.7368


In [543]:
test_review = '''Честно говоря, героиня Лядовой намного интереснее главного героя. Ее хочется смотреть. Экстравагантного, экзистенциального персонажа - самого психа -нет. Не тянет он. А проблему секса в пожилом возрасте, с которой сталкивается мать героя -выглядит откровенно странно, искусственно и даже слегка уродливо, из-за специфической внешности персонажа Розы Хайрулиной. Она больше похожа на человека из космоса, нежели на женщину с типичными 'проевропейскими' проблемами. Не резонирует. Даже не вызывает интерес. Сериал не выглядит даже как самобытный изыск. Он выглядит как технологичная попытка выйти из сериального шаблона, но оттого создающая тот самый шаблон. Претензия на 'умное' кино не оправдана. Смотрела из любопытства, но с усилием. В конце уже просто на перемотке. Не цепляет.'''

In [545]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
pred = model.predict(np.asarray(test_tokenized))
categories[np.argmax(pred[0])]

'neutral'

In [40]:
# Еще попробуем не рекуррентную
model = Sequential()
model.add(Embedding(max_features, 128, mask_zero=True, input_length=128))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy', Precision(), Recall()])

In [45]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 128, 128)          10056704  
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              16778240  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 3075      
Total params: 26,838,019
Trainable params: 26,838,019
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(x_train_pad, y_train, batch_size=50, epochs=2, 
                    validation_data=(x_test_pad, y_test))

Epoch 1/2
28/28 [==============================] - 4s 157ms/step - loss: 0.7955 - accuracy: 0.7120 - precision_6: 0.7287 - recall_6: 0.6265 - val_loss: 0.6612 - val_accuracy: 0.7398 - val_precision_6: 0.7508 - val_recall_6: 0.7310
Epoch 2/2
28/28 [==============================] - 4s 145ms/step - loss: 0.1479 - accuracy: 0.9817 - precision_6: 0.9931 - recall_6: 0.9423 - val_loss: 0.6175 - val_accuracy: 0.7602 - val_precision_6: 0.8086 - val_recall_6: 0.7164


In [691]:
# Переучивается быстро. Но вообще - валидационные метрики лучше все.

In [23]:
test_review = '''Честно говоря, героиня Лядовой намного интереснее главного героя. Ее хочется смотреть. Экстравагантного, экзистенциального персонажа - самого психа -нет. Не тянет он. А проблему секса в пожилом возрасте, с которой сталкивается мать героя -выглядит откровенно странно, искусственно и даже слегка уродливо, из-за специфической внешности персонажа Розы Хайрулиной. Она больше похожа на человека из космоса, нежели на женщину с типичными 'проевропейскими' проблемами. Не резонирует. Даже не вызывает интерес. Сериал не выглядит даже как самобытный изыск. Он выглядит как технологичная попытка выйти из сериального шаблона, но оттого создающая тот самый шаблон. Претензия на 'умное' кино не оправдана. Смотрела из любопытства, но с усилием. В конце уже просто на перемотке. Не цепляет.'''

In [694]:
test_tokenized = tokenizer.texts_to_sequences((test_review, ))
test_tokenized_pad = sequence.pad_sequences(test_tokenized, maxlen=128, padding='post')
pred = model.predict(test_tokenized_pad)
categories[np.argmax(pred[0])]

'neutral'

### Вобщем - появились такие вопросы...
1. Правильно ли я понимаю, что количество рекуррентных нейронов - это глубина рекуррентности? То-есть на примере - если у нас в модель приходит текст из 1000 слов - а в LSTM слой состоит из 64 нейронов. Что это значит? Что наш рекуррентный слой может теоретически выделять (запоминать - ну - не знаю, как правильно сформулировать - выделять как значимые признаки) фразы длинной вплоть до 64 слов?
2. Как это происходит? Допустим у нас сделан пэддинг 128. Первому нейрону в LSTM дается первое слово, второму - второе ну и так накапливается 64 состояния - с последнего мы снимаем результат (ну - финальное состояние) (у нас many-to-one например - бинарная классификация).. Но слов у нас в каждом примере больше чем 64.. (ну там нули в педдинге к тому-же) - нейронов 64 - как обрабатываются слова которые не вмещаются в количество нейронов? Или мы должны подбирать количество нейронов под потенциальную длину интересующего нас события во входных данных? Типа 32 нейрона - подаем на вход подвыборки из 32 кадров?
3. Еще вопрос - уже не по рекуррентности - у меня многоклассовая классификация - удобные мне метрики precision и recall (ну в идеале F-мера) - должны быть для каждого класса отдельно (ну чтобы понимать - насколько хорошо разделяет модель данные) - но при использовании по умолчанию - дается некие общие значения - как их понять - неизвестно. Можно ли вывести метрики для каждого класса в отдельности?